In [1]:
from openff.units import unit
from openff.evaluator.backends import ComputeResources
from openff.evaluator.backends.dask import DaskLocalCluster
from openff.evaluator.datasets import (
    MeasurementSource,
    PhysicalPropertyDataSet,
    PropertyPhase,
)
from openff.evaluator.client import EvaluatorClient, RequestOptions, Request
from openff.evaluator.server.server import Batch, EvaluatorServer

from openff.evaluator.forcefield import (
    LigParGenForceFieldSource,
    SmirnoffForceFieldSource,
    TLeapForceFieldSource,
)

from openff.evaluator.properties import Density, EnthalpyOfMixing
from openff.evaluator.substances import Substance
from openff.evaluator.thermodynamics import ThermodynamicState

## Create a toy dataset

Properties can be downloaded from the internet, loaded from a file, or created dynamically. Here we quickly create a small dataset.

In [2]:
dataset = PhysicalPropertyDataSet()
thermodynamic_state = ThermodynamicState(
    temperature=298.15 * unit.kelvin,
    pressure=101.325 * unit.kilopascal,
)
dataset.add_properties(
    Density(
        thermodynamic_state=thermodynamic_state,
        phase=PropertyPhase.Liquid,
        value=1.0 * Density.default_unit(),
        uncertainty=1.0 * Density.default_unit(),
        source=MeasurementSource(doi=" "),
        substance=Substance.from_components("CCCO"),
    ),
    EnthalpyOfMixing(
        thermodynamic_state=thermodynamic_state,
        phase=PropertyPhase.Liquid,
        value=1.0 * EnthalpyOfMixing.default_unit(),
        uncertainty=1.0 * EnthalpyOfMixing.default_unit(),
        source=MeasurementSource(doi=" "),
        substance=Substance.from_components("CCCO", "O"),
    )
)

We now need to define options for both equilibration and pre-equilibrated simulation.

In [3]:
equilibration_options = RequestOptions()
equilibration_options.calculation_layers = ["EquilibrationLayer"]
density_equilibration_schema = Density.default_equilibration_schema(
    n_molecules=256,
    relative_tolerance=0.01,
)

dhmix_equilibration_schema = EnthalpyOfMixing.default_equilibration_schema(
    n_molecules=256,
    relative_tolerance=0.01,
)
equilibration_options.add_schema(
    "EquilibrationLayer",
    "Density",
    density_equilibration_schema,
)
equilibration_options.add_schema(
    "EquilibrationLayer",
    "EnthalpyOfMixing",
    dhmix_equilibration_schema,
)

preequilibrated_simulation_options = RequestOptions()
preequilibrated_simulation_options.calculation_layers = ["PreequilibratedSimulationLayer"]
density_preequilibration_schema = Density.default_preequilibrated_simulation_schema(
    n_molecules=256,
    relative_tolerance=0.01,
)
dhmix_preequilibration_schema = EnthalpyOfMixing.default_preequilibrated_simulation_schema(
    n_molecules=256,
    relative_tolerance=0.01,
)
preequilibrated_simulation_options.add_schema(
    "PreequilibratedSimulationLayer",
    "Density",
    density_preequilibration_schema,
)
preequilibrated_simulation_options.add_schema(
    "PreequilibratedSimulationLayer",
    "EnthalpyOfMixing",
    dhmix_preequilibration_schema,
)

build_coordinates
assign_parameters
energy_minimisation
conditional_group


We load a force field from SMIRNOFF.

In [5]:
force_field_path = "openff-2.1.0.offxml"
force_field_source = SmirnoffForceFieldSource.from_path(
    force_field_path
)

In [ ]:
with DaskLocalCluster(
    number_of_workers=1,
    resources_per_worker=ComputeResources(
        number_of_threads=1,
        number_of_gpus=1,
        preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA,
    ),
) as calculation_backend:
    server = EvaluatorServer(
        calculation_backend=calculation_backend,
        working_directory=".",
        delete_working_files=False
    )
    with server:
        client = EvaluatorClient()
    
        # test equilibration
        request, error = client.request_estimate(
            dataset,
            force_field_source,
            equilibration_options,
        )

callback
[<Future: pending, key: _wrapped_function-47cfb53078386e084a75f760e4b05d48>]
<Future: pending, key: _wrapped_function-f83d9cf950709775725722e59082c548>


/Users/lily/pydev/openff-evaluator/openff/evaluator/workflow/workflow.py:942: UserWarning: executing
  warnings.warn("executing")
/Users/lily/pydev/openff-evaluator/openff/evaluator/workflow/workflow.py:946: UserWarning: dict_keys(['176316eace504875a0a82be6cd6c017c|build_coordinates_component_0', '176316eace504875a0a82be6cd6c017c|assign_parameters_component_0', '176316eace504875a0a82be6cd6c017c|energy_minimisation_component_0', '176316eace504875a0a82be6cd6c017c|conditional_group_component_0', '176316eace504875a0a82be6cd6c017c|build_coordinates_component_1', '176316eace504875a0a82be6cd6c017c|assign_parameters_component_1', '176316eace504875a0a82be6cd6c017c|energy_minimisation_component_1', '176316eace504875a0a82be6cd6c017c|conditional_group_component_1', '176316eace504875a0a82be6cd6c017c|build_coordinates_mixture', '176316eace504875a0a82be6cd6c017c|assign_parameters_mixture', '176316eace504875a0a82be6cd6c017c|energy_minimisation_mixture', '176316eace504875a0a82be6cd6c017c|conditional_gr